In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
# import en_core_web_lg

import tqdm
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nlp = spacy.load("en")

# Load Data

In [3]:
data_df = pd.read_csv('../data/bgg_data_mod.csv', index_col=0)

data_df = data_df[data_df.game_desc.str.len() > 5]

print(data_df.shape)
data_df.head()

(18693, 266)


,name,game_desc,boardgamecategorys,boardgamemechanics,game_id,CAT:,CAT:Abstract_Strategy,CAT:Action/Dexterity,CAT:Adventure,CAT:Age_of_Reason,...,mech:Variable_Phase_Order,mech:Variable_Player_Powers,mech:Variable_Setup,mech:Victory_Points_as_a_Resource,mech:Voting,mech:Worker_Placement,mech:Worker_Placement_with_Dice_Workers,mech:You_Choose,mech:Zone_of_Control,mech:and_Pool_Building
0,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Action Retrieval', 'Campaign / Battle Card D...",174430,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...",161936,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...","['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",167791,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,"['Economic', 'Industry / Manufacturing', 'Tran...","['Connections', 'Hand Management', 'Income', '...",224517,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Through the Ages: A New Story of Civilization,Through the Ages: A New Story of Civilization ...,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:...",182028,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


train_df = pd.read_csv('../data/cats/train.csv', index_col=0)
valid_df = pd.read_csv('../data/cats/valid.csv', index_col=0)
test_df = pd.read_csv('../data/cats/test.csv', index_col=0)

print(len(train_df), len(valid_df), len(test_df), len(train_df)+ len(valid_df)+ len(test_df))

train_df.head()

train_df['description'] = train_df.description.apply(lambda x: strip_tags(x))
valid_df['description'] = valid_df.description.apply(lambda x: strip_tags(x))
test_df['description'] = test_df.description.apply(lambda x: strip_tags(x))

train_df.head()

In [4]:
# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [5]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [6]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm.tqdm(data_df.description):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

100%|████████████████████████████████████████████████████████████████████████████| 18693/18693 [07:49<00:00, 39.85it/s]


In [7]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=10, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [9]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.064*"tile" + 0.051*"board" + 0.050*"piece" + 0.024*"place" + '
  '0.022*"space" + 0.019*"animal" + 0.017*"color" + 0.016*"square" + '
  '0.016*"pawn" + 0.015*"player"'),
 (1,
  '0.083*"card" + 0.080*"player" + 0.033*"game" + 0.026*"play" + 0.018*"point" '
  '+ 0.018*"  " + 0.017*"turn" + 0.014*"win" + 0.013*"number" + 0.012*"dice"'),
 (2,
  '0.061*"de" + 0.021*"la" + 0.015*"que" + 0.015*"en" + 0.014*"y" + 0.013*"el" '
  '+ 0.012*"un" + 0.009*"los" + 0.007*"se" + 0.007*"para"'),
 (3,
  '0.211*"    " + 0.013*"robot" + 0.011*"Dé" + 0.011*"circuit" + '
  '0.009*"implement" + 0.005*"Rock" + 0.005*"Formule" + 0.005*"Paper" + '
  '0.005*"Devil" + 0.005*"№"'),
 (4,
  '0.043*"game" + 0.019*"  " + 0.014*"player" + 0.008*"question" + '
  '0.008*"team" + 0.007*"answer" + 0.007*"play" + 0.007*"word" + 0.007*"try" + '
  '0.007*"board"'),
 (5,
  '0.027*"money" + 0.020*"game" + 0.017*"player" + 0.015*"buy" + 0.014*"city" '
  '+ 0.011*"sell" + 0.009*"property" + 0.009*"company" + 0.009*"count

In [10]:
# Compute Perplexity
# a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  


Perplexity:  -8.690066921572363


In [11]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, words)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.186089  0.131933       1        1  26.632496
7      0.139015  0.009319       2        1  16.975204
4      0.156244  0.042842       3        1  14.971228
8      0.065352 -0.160762       4        1  11.645700
6      0.120457 -0.135897       5        1  10.964116
9      0.095666 -0.110338       6        1   8.580413
5     -0.026041 -0.008095       7        1   4.412901
0     -0.041042  0.277155       8        1   3.760089
2     -0.348935 -0.022126       9        1   1.317792
3     -0.346805 -0.024032      10        1   0.740058, topic_info=            Term          Freq         Total Category  logprob  loglift
10          card  40922.000000  40922.000000  Default  30.0000  30.0000
97         board   8607.000000   8607.000000  Default  29.0000  29.0000
455                2987.000000   2987.000000  Default  28.0000  28.0000
283         tile   4370.000000   4370.000000  Default  27.0000  27.0000
57        player  57851.000000  57851.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
8870        Rock     71.627174    129.570816  Topic10  -5.2404   4.3134
505    implement    125.820236    339.498596  Topic10  -4.6770   3.9136
20889      Steve     27.651907     48.330490  Topic10  -6.1922   4.3478
942         gear     32.759190    182.791367  Topic10  -6.0227   3.1870
552          Han     25.602480     47.519470  Topic10  -6.2692   4.2877

[690 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
0          1  0.428839        
0          2  0.000250        
0          3  0.253280        
0          4  0.039035        
0          5  0.227772        
...      ...       ...     ...
5215       2  0.918422  zombie
5215       6  0.080672  zombie
12399      8  0.989243     zoo
5554       9  0.985715       à
40793     10  0.989651       №

[1528 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 5, 9, 7, 10, 6, 1, 3, 4])

# With TF-IDF

In [12]:
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [13]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=words,
                                           num_topics=10, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model_tfidf.print_topics())
doc_lda = lda_model_tfidf[corpus_tfidf]

[(0,
  '0.000*"Rory" + 0.000*"Wonderland" + 0.000*"telling" + 0.000*"Alice" + '
  '0.000*"Carrom" + 0.000*"librarian" + 0.000*"SonhandoSonhando" + '
  '0.000*"Brady" + 0.000*"Dés" + 0.000*"Merveilles"'),
 (1,
  '0.000*"burger" + 0.000*"Burger" + 0.000*"Cry" + 0.000*"Havoc" + '
  '0.000*"Railways" + 0.000*"cookie" + 0.000*"dino" + 0.000*"Shogi" + '
  '0.000*"mustard" + 0.000*"Ketchup"'),
 (2,
  '0.000*"Turtles" + 0.000*"turtle" + 0.000*"Créateurs" + 0.000*"Concours" + '
  '0.000*"Société" + 0.000*"Teenage" + 0.000*"Jeux" + 0.000*"Ogre" + '
  '0.000*"TMNT" + 0.000*"Ali"'),
 (3,
  '0.002*"ingredient" + 0.002*"=" + 0.002*"ghost" + 0.002*"Disney" + '
  '0.002*"memory" + 0.002*"dog" + 0.002*"judge" + 0.002*"pop" + 0.002*"brain" '
  '+ 0.002*"british"'),
 (4,
  '0.003*"$" + 0.002*"hotel" + 0.002*"monopoly" + 0.002*"frog" + 0.002*"trump" '
  '+ 0.001*"tip" + 0.001*"electronic" + 0.001*"pad" + 0.001*"mouth" + '
  '0.001*"sword"'),
 (5,
  '0.001*"ant" + 0.000*"ladybug" + 0.000*"anthill" + 0.000*

In [15]:
# Compute Perplexity
# a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus_tfidf))  


Perplexity:  -11.715382635112029


In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, words)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.386344  0.008450       1        1  85.085609
3      0.073576  0.138453       2        1   8.908816
4      0.056138 -0.067896       3        1   3.690764
6      0.046669 -0.029512       4        1   1.710932
9      0.035874 -0.009669       5        1   0.160841
0      0.034827 -0.007973       6        1   0.092847
5      0.034893 -0.008082       7        1   0.090946
2      0.034851 -0.008032       8        1   0.087929
1      0.034772 -0.007891       9        1   0.086712
8      0.034744 -0.007847      10        1   0.084604, topic_info=              Term       Freq       Total Category  logprob  loglift
10            card  654.00000  654.000000  Default  30.0000  30.0000
0                   616.00000  616.000000  Default  29.0000  29.0000
3437    ingredient   23.00000   23.000000  Default  28.0000  28.0000
4886             =   22.00000   22.000000  Default  27.0000  27.0000
497          ghost   20.00000   20.000000  Default  26.0000  26.0000
...            ...        ...         ...      ...      ...      ...
36099  Ultramarine    0.00104    0.242474  Topic10 -11.4561   1.6234
11643       Blokus    0.00104    0.265085  Topic10 -11.4561   1.5342
32253           GW    0.00104    0.250393  Topic10 -11.4561   1.5913
842          Terra    0.00104    1.388385  Topic10 -11.4561  -0.1216
11290       40,000    0.00104    2.939343  Topic10 -11.4561  -0.8717

[558 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          1  1.000429            
455        1  1.001052            
10996      1  1.067548            
8197       3  0.974180           $
4584       2  0.955116         1/2
...      ...       ...         ...
77         1  0.998038        turn
78         1  0.998965         use
166        1  1.001379         win
1420       1  0.998377        word
7954       4  1.074838        yell

[137 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 5, 7, 10, 1, 6, 3, 2, 9])